# Regression Models

In [1]:

"""Importing all the required library for Randomforest,KNN and XG boost regressor """
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from datetime import datetime

In [19]:
def log(file_objects,log_message):
    """This method will be used for logging all the information to the file."""
     
    now = datetime.now()
    date = now.date()
    time  = now.strftime("%H:%M:%S")
    file_objects.write(str(date) + "/" + str(time) + "\t\t" + log_message +"\n")

# KNN Regression Model

In [32]:
"""# Created a method, which will take a dataframe with regression problem and will apply KNN algorithm   """
def get_tuned_knn_model(dataframe):

    """seprating independent and dependent variable and coverting them into array form for fasten the conputation"""
    X = dataframe.iloc[:,1:-1].values
    y = dataframe.iloc[:,-1].values
    log(file_objects," data got splited into X(independent variable) and Y ( Taregt variable)") 
    
    #********************************************************************************************************
    
    """# Data is being devided into training and test by using sklearn train_test_split method. 
    we are keeping 75% data as training"""
    
    X_train,X_test,y_train,y_test =train_test_split(X,y,test_size = 0.25, random_state = 0)
    log(file_objects, "data featres are divided into training and test")
    
    #********************************************************************************************************
    
    """ Knn regression model fitting with hypertuning and checking the R2 score0"""    
    knn_reg_model = KNeighborsRegressor()
    pre_CV_model = knn_reg_model.fit(X_train,y_train)
    prediction = pre_CV_model.predict(X_test)
    log(file_objects," Model prediction done")
    
    #********************************************************************************************************
    
    """# Evaluation matrix r square and adjusted r square"""
    print("Evaluation matrix")
    r_squared = r2_score(y_test,prediction) # using Sk learn library  
    print("\nR square score without CV :-" ,r_squared)
    print("*"*70)
    raw_in_dataset = dataframe.shape[0] # number of observation in the dataset  
    feature_dataset = dataframe.shape[1] # number feature in the datset 
    
    adjusted_r2 = 1-(1-r_squared)*(raw_in_dataset-1)/(raw_in_dataset-feature_dataset-1) # calculating manually 
    print("Adjusted r square score without CV :-" ,adjusted_r2)
    log(file_objects," Rsqaure and adjusted R square calculated from pre optimized model ")
    
    #********************************************************************************************************
    
    """#   Hypertuning:- using Randomaizersearch CV for getting the best paramter for the model"""
    KNN_parameter = {"algorithm":["kd_tree", "brute"],
                "n_neighbors": list(range(1,30)),
                 "weights" : ["uniform","distance"],
                 "leaf_size": [5,10,15,20,25,30],
            "p": [1,2]}

    """# creating object for randomizer CV and passing parameters""" 
    KNN_Random_CV = RandomizedSearchCV(knn_reg_model,param_distributions = KNN_parameter,n_iter=10,scoring="r2",n_jobs=-1)
    
    #********************************************************************************************************
    
    # Giving data to Randon CV 
    KNN_Random_CV.fit(X_train,y_train)
    log(file_objects,"Random Search CV performed to find best parameter for the model")
    
    """getting best parameters from hupertune model and priting"""
    best_para = KNN_Random_CV.best_params_
    print("The best parameter selected by the random search CV algorithm:- ", best_para)
    
    """getting best score from hupertune model and priting"""
    best_score = KNN_Random_CV.best_score_ 
    print("The best score selected by the random search CV algorithm:- ",best_score)
    
    #********************************************************************************************************
    
    """fitting KNN model again with hyper tuned parameters"""
    hypertune_knn_reg_model = KNeighborsRegressor(n_neighbors = best_para["n_neighbors"],weights=best_para["weights"],
    algorithm=best_para["algorithm"],leaf_size=best_para["leaf_size"],p=best_para["p"])
    log(file_objects,"Tuned model is run successfully")
   
    # providing trainig data to model
    model_post_CV = hypertune_knn_reg_model.fit(X_train,y_train)
    
    # making prediction on test data point 
    prediction_post_CV = model_post_CV.predict(X_test)

    #********************************************************************************************************
    
    # Evaluation matrix r square and adjusted r square
    print("Evalution matrix--- chekcing the Rquare and adjusted Rsquare on tuned model ")
    r_square_post_CV = r2_score(y_test,prediction_post_CV) # using Sk learn library  
    print("\nR square score post Random CV :-" ,r_square_post_CV)
    print("*"*70)
    adjusted_r2 = 1-(1-r_square_post_CV)*(raw_in_dataset-1)/(raw_in_dataset-feature_dataset-1) # calculating manually 
    print("Adjusted r square score without CV:-" ,adjusted_r2)
    log(file_objects," Rsqaure and adjusted R square calculated from optimized model")

# Random Forest Regressor Model

In [33]:
# Created a method, which will take a dataset with regression problem and will apply Randomforest algoright 
def get_tuned_Randomforest_model(dataframe):

#     seprating independent and dependent variable and coverting them into array form for fasten the conputation
    X = dataframe.iloc[:,1:-1].values
    y = dataframe.iloc[:,-1].values
    log(file_objects," data got splited into X(independent variable) and Y ( Taregt variable)") 
    #********************************************************************************************************
    # Data is being devided into training and test by using sklearn train_test_split method. we are keeping 75% data as training
    X_train,X_test,y_train,y_test =train_test_split(X,y,test_size = 0.25, random_state = 0)
    log(file_objects, "data featres are divided into training and test")
    
    #********************************************************************************************************
#     Randomforest regression model is fitting and checking the R2 score
    Randomforest_reg_model = RandomForestRegressor()
    pre_CV_model = Randomforest_reg_model.fit(X_train,y_train)
    prediction = pre_CV_model.predict(X_test)
    log(file_objects," Model prediction done")
    
    #********************************************************************************************************
    
    # Evaluation matrix r square and adjusted r square are being calculated
    print("Evaluation matrix")
    r_squared = r2_score(y_test,prediction) # using Sk learn library  
    print("\nR square score without CV :-" ,r_squared)
    print("*"*70)
    raw_in_dataset = dataframe.shape[0] # number of observation in the dataset  
    feature_dataset = dataframe.shape[1] # number feature in the datset 
    
    adjusted_r2 = 1-(1-r_squared)*(raw_in_dataset-1)/(raw_in_dataset-feature_dataset-1) # calculating manually 
    print("Adjusted r square score without CV :-" ,adjusted_r2)
    log(file_objects," Rsqaure and adjusted R square calculated from pre optimized model ")
    
    #********************************************************************************************************
    
#   Hypertuning:- using Randomaizersearch CV for getting the best paramter for the model and defing parameter for the same to check
    Randomforest_parameter = {'n_estimators':[100,200,300,400,500],
                  'criterion' : ['mse','mae'],
                  'max_depth': [3, 5, 10, None],
                  'min_samples_split': [2,5,10,15,20,25,30,35,40,45,50],
                   "min_samples_leaf":[1,2,3,4,5]} 

    # creating object for randomizer CV and passing parameters 
    Randomforest_Random_CV = RandomizedSearchCV(Randomforest_reg_model,param_distributions = Randomforest_parameter,n_iter=10,scoring="r2",n_jobs=-1)
    log(file_objects,"Randomizer run successfully")
    
    #********************************************************************************************************
    # Giving data to Randon CV 
    Randomforest_Random_CV.fit(X_train,y_train)
    log(file_objects,"Random Search CV performed to find best parameter for the model")
    
     # getting best parameters from hupertune model and priting
    best_para = Randomforest_Random_CV.best_params_
    print("The best parameter selected by the random search CV algorithm:- ", best_para)
    
    # getting best score from hupertune model and priting
    best_score = Randomforest_Random_CV.best_score_ 
    print("\nThe best score selected by the random search CV algorithm:- ",best_score)
    
    #********************************************************************************************************
    
    # fitting KNN model again with hyper tuned parameters
    hypertune_Randomforest_reg_model = RandomForestRegressor(n_estimators = best_para["n_estimators"],criterion=best_para["criterion"],
    max_depth = best_para['max_depth'],min_samples_split = best_para['min_samples_split'],min_samples_leaf =  best_para['min_samples_leaf'], random_state = 0)
    log(file_objects,"Tuned model is run successfully")
   
    # providing trainig data to model
    model_post_CV = hypertune_Randomforest_reg_model.fit(X_train,y_train)
    log(file_objects,"Model fitting with optimized parameters")
    
    # making prediction on test data point 
    prediction_post_CV = model_post_CV.predict(X_test)
    
    #********************************************************************************************************
    
    # Evaluation matrix r square and adjusted r square
    print("\nEvalution matrix--- chekcing the Rquare and adjusted Rsquare on tuned model ")
    r_square_post_CV = r2_score(y_test,prediction_post_CV) # using Sk learn library  
    print("\nR square score post Random CV :-" ,r_square_post_CV)
    print("*"*70)
    adjusted_r2 = 1-(1-r_square_post_CV)*(raw_in_dataset-1)/(raw_in_dataset-feature_dataset-1) # calculating manually 
    print("Adjusted r square score without CV:-" ,adjusted_r2)
    log(file_objects," Rsqaure and adjusted R square calculated from optimized model")

# XGBoost Regressor

In [35]:
# Created a method, which will take a dataset with regression problem and will apply XGBoost algoright 
def get_tuned_XGBoost_model(dataframe):

#     seprating independent and dependent variable and coverting them into array form for fasten the conputation
    X = dataframe.iloc[:,1:-1].values
    y = dataframe.iloc[:,-1].values
    log(file_objects," data got splited into X(independent variable) and Y ( Taregt variable)")
    
    #********************************************************************************************************
    
    # Data is being devided into training and test by using sklearn train_test_split method. we are keeping 75% data as training
    X_train,X_test,y_train,y_test =train_test_split(X,y,test_size = 0.25, random_state = 0)
    log(file_objects, "data featres are divided into training and test")
    
    #********************************************************************************************************
    
#     XGB regression model fitting with hypertuning and checking the R2 score
    XGBoost_reg_model = xgb.XGBRegressor()
    pre_CV_model = XGBoost_reg_model.fit(X_train,y_train)
    prediction = pre_CV_model.predict(X_test)
    log(file_objects," Model prediction done")
    
    #********************************************************************************************************
    
    # Evaluation matrix r square and adjusted r square
    print("Evaluation matrix")
    r_squared = r2_score(y_test,prediction) # using Sk learn library  
    print("\nR square score without CV :-" ,r_squared)
    print("*"*70)
    raw_in_dataset = dataframe.shape[0] # number of observation in the dataset  
    feature_dataset = dataframe.shape[1] # number feature in the datset 
    
    adjusted_r2 = 1-(1-r_squared)*(raw_in_dataset-1)/(raw_in_dataset-feature_dataset-1) # calculating manually 
    print("Adjusted r square score without CV :-" ,adjusted_r2)
    log(file_objects," Rsqaure and adjusted R square calculated from pre optimized model ")
    
    #********************************************************************************************************
    
#   Hypertuning:- using Randomaizersearch CV for getting the best paramter for the model and defing parameter for the same to check
    XGBoost_parameter = {"colsample_bytree" : np.linspace(0.5, 0.9, 5),
            'n_estimators':[100, 200,300,500,],
            'max_depth': [10, 15, 20, 25],
            "learning_rate":[0.1,0.3,0.5],
            "reg_alpha": list(range(1,10))} 

    # creating object for randomizer CV and passing parameters 
    XGBoost_Random_CV = RandomizedSearchCV(XGBoost_reg_model,param_distributions = XGBoost_parameter,n_iter=10,scoring="r2",n_jobs=-1)
    log(file_objects,"Randomizer run successfully")

    #********************************************************************************************************
    
    # Giving data to Randon CV 
    XGBoost_Random_CV.fit(X_train,y_train)
    log(file_objects,"Random Search CV performed to find best parameter for the model")
    
     # getting best parameters from hupertune model and priting
    best_para = XGBoost_Random_CV.best_params_
    print("The best parameter selected by the random search CV algorithm:- ", best_para)
    
    # getting best score from hupertune model and priting
    best_score = XGBoost_Random_CV.best_score_ 
    print("\nThe best score selected by the random search CV algorithm:- ",best_score)
    
    #********************************************************************************************************
    
    # fitting XGB model again with hyper tuned parameters
    hypertune_XGBoost_reg_model = xgb.XGBRegressor(n_estimators= best_para["n_estimators"],colsample_bytree = best_para["colsample_bytree"],
    max_depth= best_para["max_depth"],learning_rate = best_para["learning_rate"],reg_alpha = best_para["reg_alpha"],random_state = 0)
    log(file_objects,"Tuned model is run successfully")
   
    #********************************************************************************************************    

    # providing trainig data to model
    model_post_CV = hypertune_XGBoost_reg_model.fit(X_train,y_train)
    log(file_objects,"Model fitting with optimized parameters")
    
    # making prediction on test data point 
    prediction_post_CV = model_post_CV.predict(X_test)
    
    #********************************************************************************************************
    
    # Evaluation matrix r square and adjusted r square
    print("\nEvalution matrix--- chekcing the Rquare and adjusted Rsquare on tuned model ")
    r_square_post_CV = r2_score(y_test,prediction_post_CV) # using Sk learn library  
    print("\nR square score post Random CV :-" ,r_square_post_CV)
    print("*"*70)
    adjusted_r2 = 1-(1-r_square_post_CV)*(raw_in_dataset-1)/(raw_in_dataset-feature_dataset-1) # calculating manually 
    print("Adjusted r square score without CV:-" ,adjusted_r2)
    log(file_objects," Rsqaure and adjusted R square calculated from optimized model")

In [30]:
file_objects = open("XGBoostReg.txt","w")
get_tuned_XGBoost_model(data)
file_objects.close()

Evaluation matrix

R square score without CV :- 0.7669102001278046
**********************************************************************
Adjusted r square score without CV :- 0.7597747980909006


C:\Users\HP\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


The best parameter selected by the random search CV algorithm:-  {'reg_alpha': 2, 'n_estimators': 300, 'max_depth': 25, 'learning_rate': 0.1, 'colsample_bytree': 0.9}

The best score selected by the random search CV algorithm:-  0.8928956604799136

Evalution matrix--- chekcing the Rquare and adjusted Rsquare on tuned model 

R square score post Random CV :- 0.7404350076147475
**********************************************************************
Adjusted r square score without CV:- 0.7324891405009132


In [31]:
file_objects = open("RandomforestReg.txt","w")
get_tuned_Randomforest_model(data)
file_objects.close()

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Evaluation matrix

R square score without CV :- 0.802226859411818
**********************************************************************
Adjusted r square score without CV :- 0.7961725795978941


C:\Users\HP\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


The best parameter selected by the random search CV algorithm:-  {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 3, 'max_depth': 5, 'criterion': 'mse'}

The best score selected by the random search CV algorithm:-  0.8255845538442256

Evalution matrix--- chekcing the Rquare and adjusted Rsquare on tuned model 

R square score post Random CV :- 0.7374313349185562
**********************************************************************
Adjusted r square score without CV:- 0.7293935186405528


In [26]:
file_objects = open("KNNReg.txt","w")
get_tuned_knn_model(data)
file_objects.close()

Evaluation matrix

R square score without CV :- 0.4616380924610112
**********************************************************************
Adjusted r square score without CV :- 0.44515762590369523


C:\Users\HP\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


The best parameter selected by the random search CV algorithm:-  {'weights': 'uniform', 'p': 1, 'n_neighbors': 5, 'leaf_size': 15, 'algorithm': 'kd_tree'}
The best score selected by the random search CV algorithm:-  0.5835518290709706
Evalution matrix--- chekcing the Rquare and adjusted Rsquare on tuned model 

R square score post Random CV :- 0.539579412802726
**********************************************************************
Adjusted r square score without CV:- 0.5254849050313809
